# Loading the data
* both source and destinations are random.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/major-project/

/content/drive/MyDrive/Colab Notebooks/major-project


In [3]:
import random
import json
import os

def load_data():
    if os.path.exists('dataset/data_list_random.json'):
      with open('dataset/data_list_random.json', 'r') as f:
        d = json.load(f)
    elif os.path.exists('/content/drive/MyDrive/Colab Notebooks/major-project/dataset/data_list_random.json'):
        with open('/content/drive/MyDrive/Colab Notebooks/major-project/dataset/data_list_random.json', 'r') as f:
          d = json.load(f)
    else:
      raise Exception("data file does not exist")

    maze = d['maze']
    data = d['data']
    '''
    model data is 2D list
    inner dimension representing each data item: [0.52, 0.5, 0.1522936964597336, 'right']
    '''
    # shuffle data
    random.shuffle(data)
    data[:10]
    return data

data = load_data()
data[:10]

[[0.48, 0.5, 0.20645980949989537, 'left'],
 [0.48, 0.5, 0.54163159792654, 'left'],
 [0.5, 0.52, 0.2139156191712158, 'down'],
 [0.48, 0.5, 0.10014771788387872, 'left'],
 [0.52, 0.5, 0.32644162165206925, 'right'],
 [0.5, 0.52, 0.2253323652387271, 'down'],
 [0.52, 0.5, 0.25777096569908114, 'right'],
 [0.52, 0.5, 0.3584742761392142, 'right'],
 [0.52, 0.5, 0.313711827098512, 'right'],
 [0.48, 0.5, 0.16795282873486816, 'left']]

## Training the model:
* each data item is value like: [0.5, 0.52, 0.02503692947096968, 'down']
* first two values represent vectorized co-ordinates
* third value represent distance between current location and destination
* last value represent label for model output

# Model1: LogisticRegression

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import pickle

# # Assuming your data is a list of lists
# # Each data item is of the form: [x1, x2, distance, label]
# # Example data
# data = [
#     [0.5, 0.52, 0.02503692947096968, 'down'],
#     # Add more data items here
# ]

# Extract features (first three values) and labels
X = np.array([item[:3] for item in data])
y = np.array([item[3] for item in data])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Example usage to predict a new instance
new_instance = np.array([0.6, 0.48, 0.030012, 'up'][:3])
prediction = model.predict([new_instance])
print(f"Prediction: {prediction}")

# Save the model
with open('models/logistic_regression_model_random.pkl','wb') as f:
    pickle.dump(model, f)

Accuracy: 1.0
Prediction: ['right']


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
# Loading the model
import pickle
with open("models/logistic_regression_model_random.pkl",'rb') as f:
  loaded_model = pickle.load(f)

# Example usage to predict a new instance
new_instance = np.array([0.6, 0.48, 0.030012, 'up'][:3])
prediction = loaded_model.predict([new_instance])
print(f"Prediction: {prediction}")


Prediction: ['right']


# Model2: DecisionTreeClassifier

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle

# Assuming your data is a list of lists
# Each data item is of the form: [x1, x2, distance, label]
data = load_data()
# # Example data
# data = [
#     [0.5, 0.52, 0.02503692947096968, 'down'],
#     # Add more data items here
# ]

# Extract features (first three values) and labels
X = np.array([item[:3] for item in data])
y = np.array([item[3] for item in data])

# Convert string labels to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree model
model = DecisionTreeClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Example usage to predict a new instance
new_instance = np.array([[0.6, 0.48, 0.030012, 'up']])
# Convert string label to numeric value
new_instance[:, 3] = label_encoder.transform(new_instance[:, 3])
prediction = model.predict(new_instance[:, :3])
print(f"Prediction: {prediction}")

# Save the model
with open("models/decision_tree_model_random.pkl",'wb') as f:
  pickle.dump(model, f)


Accuracy: 1.0
Prediction: [2]


In [15]:
# Loading the model
import pickle
with open("models/decision_tree_model_random.pkl",'rb') as f:
  loaded_model = pickle.load(f)

# Example usage to predict a new instance
new_instance = np.array([0.6, 0.48, 0.030012, 'up'][:3])
prediction = loaded_model.predict([new_instance])
print(f"Prediction: {prediction}")


Prediction: [2]
